In [ ]:
from tredictpy import tredict
import time
import os
import json
import psutil
import sys

In [ ]:
json_db = "./zwift_to_tredict.json"
in_progress = "inProgressActivity.fit"
activity_dir = os.path.expanduser("~/Zwift/Activities/")
upload_past_activities = False

In [ ]:
client = tredict.TredictPy()

In [ ]:
# First run, need to authorise and get an access token
# Or was run before but did not complete authorisation
if not client.is_authorised():
    client.request_auth_code()
    client.request_user_access_token()

In [ ]:
# An access token was obtained before but it has expired
# can refresh using the refresh token
if not client.is_user_access_token_valid():
    client.request_user_access_token(refresh=True)

In [ ]:
if not os.path.exists(json_db):

    # Get a list of present activities
    activities = [
        dict(
            [
                ("activity", activity),
                ("uploaded", False),
                ("already_present", True)
            ]
        ) for activity in os.listdir(activity_dir) if activity.endswith(".fit") and activity != in_progress]

    # Upload past activities if set
    if upload_past_activities:
        for activity in activities:
            client.activity_upload(f"{activity_dir}{activity['activity']}")
            activity["uploaded"] = True

    last_checked = int(time.time())

    # Write to file to keep track
    with open(json_db, "wt") as f:
        f.write(
            json.dumps(
                {
                    "last_checked": last_checked,
                    "activities": activities,
                },
                indent=4
            )
        )


In [ ]:
with open(json_db, "rt") as f:
    activities = json.loads(f.read())

In [ ]:
print("Launching Zwift...")
zwift = psutil.subprocess.run("zwift")

In [ ]:
if zwift.returncode != 0:
    print("Zwift failed to launch!")
    sys.exit()

In [ ]:
while True:
    if not [p.info["name"] for p in psutil.process_iter(["name"]) if "zwift" in p.info["name"].lower()]:
        print("Zwift exited!")
        break
    else:
        time.sleep(10)

In [ ]:
# Check for new activities against old activities
new_activities = [
    activity for activity in os.listdir(activity_dir)
    if activity not in [a["activity"] for a in activities["activities"]] and
    activity != in_progress
]

In [ ]:
for activity in new_activities:

    # Upload new activities
    client.activity_upload(f"{activity_dir}{activity}")

    # Add to the record
    activities["activities"].append(
        {
            "activity": activity,
            "uploaded": True,
            "already_present": False,
        }
    )

In [ ]:
activities["last_checked"] = int(time.time())

In [ ]:
with open(json_db, "wt") as f:
    f.write(
        json.dumps(activities, indent=4)
    )